In [3]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
%%appyter markdown
# {{ StringField(name='title', label='Title', default='My Title').render_value }}

{{ TextField(name='description', label='Description', default='My description').render_value }}

In [ ]:
%%appyter code_exec

# if small molecule name inputted, will go through its CSV file and sort by target affinity
import pandas as pd

sm_name = {{ StringField(
        name = 'Small molecule name',
        label = 'Small molecule name',
        description = 'The small molecule name',
        default = 'Afatinib'
)}}

kinome_df = pd.read_excel('http://lincs.hms.harvard.edu/wordpress/wp-content/uploads/2013/11/HMS-LINCS_KinomeScan_Datasets_2018-01-18.xlsx')
kinome_df['sm_hms_id'] = kinome_df['sm_hms_id'].str.replace(r'\D', '') #remove HMSL before the ID
hms_id = kinome_df.loc[kinome_df['sm_name'] == sm_name, 'sm_hms_id']

# remove the row number
hms_id_list = []
for hms_id in hms_id:
    hms_id_list.append(hms_id)
    
hms_id_string = str(hms_id_list[0])
old_url = 'http://lincs.hms.harvard.edu/db/datasets/20000/results?small+molecules=HMS_ID&output_type=.csv'
new_url = old_url.replace("HMS_ID", hms_id_string) # fills in HMS_ID with the correct small molecule 
data = pd.read_csv(new_url)
df = pd.DataFrame(data)

target_affinity_1 = df.loc[df['Binding Class'] == 1, 'HUGO Gene Symbol']
target_affinity_2 = df.loc[df['Binding Class'] == 2, 'HUGO Gene Symbol']
target_affinity_3 = df.loc[df['Binding Class'] == 3, 'HUGO Gene Symbol']
target_affinity_10 = df.loc[df['Binding Class'] == 10, 'HUGO Gene Symbol'] 

#js = Javascript('alert("Invalid input")')
kinase1_list = []
kinase2_list = []
kinase3_list = []

for kinase in target_affinity_1:
    if kinase not in kinase1_list: # removes duplicates
        kinase1_list.append(str(kinase))

if len(kinase1_list) != 0:
    print(sm_name + ' binds to the following kinases with Kd < 100 nM: ')
    kinase1_string = ', '.join(kinase1_list)
    print(kinase1_string)
    print()

for kinase in target_affinity_2:
    if kinase not in kinase2_list: # removes duplicates
        kinase2_list.append(str(kinase))

if len(kinase2_list) != 0:
    print(sm_name + ' binds to the following kinases with 100 nM ≤ Kd < 1µM: ')
    kinase2_string = ', '.join(kinase2_list)
    print(kinase2_string)
    print()

for kinase in target_affinity_3:
    if kinase not in kinase3_list: # removes duplicates
        kinase3_list.append(str(kinase))

if len(kinase3_list) != 0:
    print(sm_name + " binds to the following kinases with 1µM ≤ Kd < 10 µM: ")
    kinase3_string = ', '.join(kinase3_list)
    print(kinase3_string)

In [ ]:
# Create bar graph for visualizing the kinases

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

data_plot = {'Equilibrium Dissociation Constant':  ['Kd < 100 nM','100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM'], 
             'Number of kinases': [len(kinase1_list), len(kinase2_list), len(kinase3_list)] }

df_plot = pd.DataFrame (data_plot, columns = ['Equilibrium Dissociation Constant', 'Number of kinases'])

print(df_plot)

# Draw the bar graph and format it
bar = df_plot.plot(kind='bar',figsize=(20, 12), color = "blue", legend = None, x = 'Equilibrium Dissociation Constant', alpha = 0.5)
plt.xlabel('Equilibrium Dissociation Constant', fontsize = 25)
plt.xticks(rotation = 360, fontsize = 20)
plt.yticks(fontsize = 15)
plt.ylabel('Number of kinases', fontsize = 25)
plt.title('Kinases bound by ' + sm_name, fontsize = 35)

kinase1_new = []
kinase2_new = []
kinase3_new = []

print(kinase3_list)

if (len(kinase1_list) > 29):
    kinase1_new = list(kinase1_list[:19])
    kinase1_new.append('...')
    kinase1_new.append('See table')
else:
    kinase1_new = kinase1_list

if (len(kinase2_list) > 20):
    kinase2_new = list(kinase2_list[:19])
    kinase2_new.append('...')
    kinase2_new.append('See table')
else:
    kinase2_new = kinase2_list 
    
if (len(kinase3_list) > 20):
    kinase3_new = list(kinase3_list[:19])
    kinase3_new.append('...')
    kinase3_new.append('See table')
else:
    kinase3_new = kinase3_list


kinase1_label = '\n'.join(kinase1_new)
kinase2_label = '\n'.join(kinase2_new)
kinase3_label = '\n'.join(kinase3_new)

rects = bar.patches
labels = [kinase1_label, kinase2_label, kinase3_label]

#for rect, label in zip(rects, labels):
 #   height = rect.get_height()
  #  bar.text(rect.get_x() + rect.get_width() / 2, height + 2, label,
   #         ha='center', va='bottom')

# Label each bar
label_num = 0
kin_lab = 'kinaseNUM_len'

for rect, label in zip(rects, labels):
    # Get X and Y placement of label from rect.
    x_value = rect.get_x() + rect.get_width() / 2
    
    # Gets correct length of the kinase list to determine Y placement 
    label_num += 1 
    label_len = kin_lab.replace('NUM', str(label_num))
    my_dict = {}
    my_dict['kinase1_len'] = len(kinase1_list)
    my_dict['kinase2_len'] = len(kinase2_list)
    my_dict['kinase3_len'] = len(kinase3_list)
    print(my_dict[label_len])
    y_value = my_dict[label_len] / 20

    # Number of points between bar and label. Change to your liking.
    space = 0

    # Create annotation
    bar.annotate(
            label,                      
            (x_value, y_value),         # Place label 
            xytext=(0, space),          # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally centered
            va='bottom',
            size = 20) 


data_table = {'Equilibrium Dissociation Constant':  ['Kd < 100 nM','100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM'], 
             'Kinases': [kinase1_list, kinase2_list, kinase3_list] }

dt = pd.DataFrame (data_plot, columns = ['Equilibrium Dissociation Constant', 'Kinases'])

print(dt)   

In [ ]:
import plotly.graph_objects as go
import textwrap 

x = ['Kd < 100 nM', '100 nM ≤ Kd < 1µM', '1µM ≤ Kd < 10 µM']
y = [len(kinase1_list), len(kinase2_list), len(kinase3_list)]

kinase1_lab = '<br>'.join(textwrap.wrap(kinase1_string, width=50))
kinase2_lab ='<br>'.join(textwrap.wrap(kinase2_string, width=50))
kinase3_lab ='<br>'.join(textwrap.wrap(kinase2_string, width=50))

fig = go.Figure(data=[go.Bar(x = x, y = y, text = y, textposition = 'auto',
                             hovertext = [kinase1_lab, kinase2_lab, kinase3_lab], 
                             hoverlabel = dict(font = dict(size = 18)),
                             marker={'color': y,
                                               'colorscale': 'Plotly3'})])

fig.update_layout(
    title = {
        'text': 'Kinases bound by ' + sm_name,
        'y':0.87,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
    xaxis_title = "Equilibrium Dissociation Constant",
    yaxis_title = "Number of kinases",
    font = dict(
        #family = "Courier New, monospace",
        size = 18,
        color = 'black'
    )
)

fig.show()


In [ ]:
%%appyter code_exec

import pickle

# if kinase inputted, return small molecules  
kinase_name = {{ StringField(
        name = 'Kinase name',
        label = 'Kinase name',
        description = 'The kinase name',
        default = 'ABL1'
)}}

# retrieve the dictionaries and unpickle
with open('dict_1.pickle', 'rb') as handle:
    dict_1 = pickle.load(handle)
with open('dict_2.pickle', 'rb') as handle:
    dict_2 = pickle.load(handle)
with open('dict_3.pickle', 'rb') as handle:
    dict_3 = pickle.load(handle)
# lists for each target affinity for the small molecules
target_aff1 = []
target_aff2 = []
target_aff3 = []

if kinase_name in dict_1:
    target_aff1 = dict_1[kinase_name]
if kinase_name in dict_2:
    target_aff2 = dict_2[kinase_name]
if kinase_name in dict_3:
    target_aff3 = dict_3[kinase_name]
    
if len(target_aff1) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with Kd < 100 nM: ')
    target1_string = ', '.join(target_aff1)
    print(target1_string)
    print()
    
if len(target_aff2) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with 100 nM ≤ Kd < 1µM: ')
    target2_string = ', '.join(target_aff2)
    print(target2_string)    
    print()
    
if len(target_aff3) != 0:
    print('Small molecules that bind to ' + kinase_name + ' with 1µM ≤ Kd < 10 µM: ')
    target3_string = ', '.join(target_aff3)
    print(target3_string)    
    print()